In [2]:
import pandas as pd
import os
import sweetviz as sv
import geopandas as gpd
from shapely import wkt
import contextily as ctx
import matplotlib.pyplot as plt
import seaborn as sns
import geodatasets
from shapely.geometry import Point
from geopy.geocoders import Nominatim
import warnings
import plotly.express as px
from geopy.geocoders import ArcGIS
import geopandas as gpd
import re
warnings.filterwarnings('ignore')

In [3]:
os.chdir('C:/Users/edwin/OneDrive/Escritorio/Personal files/Courses/Semester 2/Project Management/Head Competition/Code/Datasets')

In [4]:
os.listdir()

['20240117-2024-Facilities-Details-FCI.xlsx',
 'city_451_BCCAQv2_RCP85_annual_plus30_ensemble.csv',
 'CoH-CFEM-2023-Facilities-Details.xlsx',
 'facilities_wlocation.csv',
 'Metadata',
 'Ward_Boundaries.geojson']

In [16]:
df=pd.read_excel("20240117-2024-Facilities-Details-FCI.xlsx").loc[1:,:]
df.columns=df.iloc[0]
df = df.drop(1)
df.head()

1,Asset Name,Asset Type,Asset Name: Address,Asset Name: Size,Asset Name: Measure Unit,Asset Name: Date Built,Custom: Assessment Date,Asset Name: Current Replacement Value,Current Needs Value,Asset Name: Current FCI
2,703 HIGHWAY 8 (ALECTRA UTILITIES),Work Yards & Maintenance,703 HIGHWAY 8,31411,Sq.Ft.,1986-04-26,Model,12828450.29,972839.51,0.075835
3,ALEXANDER PARK - FIELDHOUSE,Park Facilities - Public Use,201 WHITNEY AVENUE,1954,Sq.Ft.,1973-01-02,2017,395826.67,39145.35,0.098895
4,ALEXANDER PARK WADING POOL UTILITY BUILDING,Outdoor Pools,201 WHITNEY AVENUE,72,Sq.Ft.,2011-01-01,2017,14585.22,0,0
5,ANCASTER AQUATIC CENTRE (P),Indoor Pools,47 MEADOWBROOK DRIVE,19067,Sq.Ft.,1982-01-02,2020,13716984.75,304172.85,0.022175
6,ANCASTER COMMUNITY CENTRE PARK - FIELDHOUSE,Park Facilities - Public Use,385 JERSEYVILLE ROAD WEST,1112,Sq.Ft.,1996-01-02,2011,225260.62,18695.65,0.082996


In [18]:
df_location=df[["Asset Name","Asset Name: Address","Asset Name: Size","Asset Name: Date Built","Custom: Assessment Date"]]
df_location

1,Asset Name,Asset Name: Address,Asset Name: Size,Asset Name: Date Built,Custom: Assessment Date
2,703 HIGHWAY 8 (ALECTRA UTILITIES),703 HIGHWAY 8,31411,1986-04-26,Model
3,ALEXANDER PARK - FIELDHOUSE,201 WHITNEY AVENUE,1954,1973-01-02,2017
4,ALEXANDER PARK WADING POOL UTILITY BUILDING,201 WHITNEY AVENUE,72,2011-01-01,2017
5,ANCASTER AQUATIC CENTRE (P),47 MEADOWBROOK DRIVE,19067,1982-01-02,2020
6,ANCASTER COMMUNITY CENTRE PARK - FIELDHOUSE,385 JERSEYVILLE ROAD WEST,1112,1996-01-02,2011
...,...,...,...,...,...
494,WOODLAND CEMETERY EQUIPMENT STORAGE,700 SPRING GARDENS ROAD,1200,1923-04-02,NaN
495,WOODLAND CEMETERY SOIL STORAGE,700 SPRING GARDENS ROAD,600,2021-11-08,NaN
496,Woodlands Park - Concession / Washrooms,501 Barton St. E.,2495,1966-01-02,2020
497,Woodward Park - Washroom / Sun Shelter,589 Woodward Ave.,667,1999-01-02,Audit


In [26]:
def row_cleaning(row):
    address_nf=row["Asset Name: Address"]
    if (address_nf=="") or (address_nf==None):
        pass
    else:
        address=address_nf.lower()+", Hamilton, Ontario"
    return address

In [27]:
df_location["formattted_address"]= df_location.apply(lambda row:row_cleaning(row),axis=1)
df_location.head()

1,Asset Name,Asset Name: Address,Asset Name: Size,Asset Name: Date Built,Custom: Assessment Date,formattted_address
2,703 HIGHWAY 8 (ALECTRA UTILITIES),703 HIGHWAY 8,31411,1986-04-26,Model,"703 highway 8, Hamilton, Ontario"
3,ALEXANDER PARK - FIELDHOUSE,201 WHITNEY AVENUE,1954,1973-01-02,2017,"201 whitney avenue, Hamilton, Ontario"
4,ALEXANDER PARK WADING POOL UTILITY BUILDING,201 WHITNEY AVENUE,72,2011-01-01,2017,"201 whitney avenue, Hamilton, Ontario"
5,ANCASTER AQUATIC CENTRE (P),47 MEADOWBROOK DRIVE,19067,1982-01-02,2020,"47 meadowbrook drive, Hamilton, Ontario"
6,ANCASTER COMMUNITY CENTRE PARK - FIELDHOUSE,385 JERSEYVILLE ROAD WEST,1112,1996-01-02,2011,"385 jerseyville road west, Hamilton, Ontario"


In [28]:
#nom=ArcGIS()

In [30]:
#example=nom.geocode('703 highway 8, Hamilton, Ontario')
#example

Location(703 Hwy 8, Stoney Creek, Ontario, L8E 5J6, (43.212915315858, -79.698300482565, 0.0))

In [32]:
def get_lat_lng(row):
    address=row["formattted_address"]
    try: 
        location=nom.geocode(address) 
        latitude=location.latitude
        longitude=location.longitude
        formatted_address=location.address
    except:
        location=None
        latitude=None
        longitude=None
        formatted_address=None
    return (latitude,longitude,formatted_address,location)

In [33]:
df_location[['latitude', 'longitude','formatted_address','location']]=df_location.apply(lambda row:get_lat_lng(row),axis=1, result_type='expand')
df_location.head()

1,Asset Name,Asset Name: Address,Asset Name: Size,Asset Name: Date Built,Custom: Assessment Date,formattted_address,latitude,longitude,formatted_address,location
2,703 HIGHWAY 8 (ALECTRA UTILITIES),703 HIGHWAY 8,31411,1986-04-26,Model,"703 highway 8, Hamilton, Ontario",43.212915,-79.698300,"703 Hwy 8, Stoney Creek, Ontario, L8E 5J6","(703 Hwy 8, Stoney Creek, Ontario, L8E 5J6, (4..."
3,ALEXANDER PARK - FIELDHOUSE,201 WHITNEY AVENUE,1954,1973-01-02,2017,"201 whitney avenue, Hamilton, Ontario",43.252779,-79.927670,"201 Whitney Ave, Hamilton, Ontario, L8S 2G6","(201 Whitney Ave, Hamilton, Ontario, L8S 2G6, ..."
4,ALEXANDER PARK WADING POOL UTILITY BUILDING,201 WHITNEY AVENUE,72,2011-01-01,2017,"201 whitney avenue, Hamilton, Ontario",43.252779,-79.927670,"201 Whitney Ave, Hamilton, Ontario, L8S 2G6","(201 Whitney Ave, Hamilton, Ontario, L8S 2G6, ..."
5,ANCASTER AQUATIC CENTRE (P),47 MEADOWBROOK DRIVE,19067,1982-01-02,2020,"47 meadowbrook drive, Hamilton, Ontario",43.215197,-80.006172,"47 Meadowbrook Dr, Ancaster, Ontario, L9G 4Y3","(47 Meadowbrook Dr, Ancaster, Ontario, L9G 4Y3..."
6,ANCASTER COMMUNITY CENTRE PARK - FIELDHOUSE,385 JERSEYVILLE ROAD WEST,1112,1996-01-02,2011,"385 jerseyville road west, Hamilton, Ontario",43.216734,-80.007322,"385 Jerseyville Rd W, Ancaster, Ontario, L9G 3L5","(385 Jerseyville Rd W, Ancaster, Ontario, L9G ..."


In [34]:
# Cargar el GeoJSON de los wards
gdf_wards = gpd.read_file('Ward_Boundaries.geojson')

In [36]:
geometry = [Point(xy) for xy in zip(df_location['longitude'], df_location['latitude'])]
gdf_coords = gpd.GeoDataFrame(df_location, geometry=geometry)

In [37]:
# Asumimos que el CRS de tus datos es WGS84 (el más común para latitud/longitud)
gdf_coords.crs = 'epsg:4326'

# Si el CRS de gdf_wards es diferente, realiza una transformación de CRS
gdf_coords = gdf_coords.to_crs(gdf_wards.crs)

In [38]:
join = gpd.sjoin(gdf_coords, gdf_wards, how="left", op="within")
join

,Asset Name,Asset Name: Address,Asset Name: Size,Asset Name: Date Built,Custom: Assessment Date,formattted_address,latitude,longitude,formatted_address,location,...,index_right,OBJECTID,WARD,COUNCILLOR_NAME,PHONE,EMAIL,WEBSITE,PICTURE_URL,Shape__Area,Shape__Length
2,703 HIGHWAY 8 (ALECTRA UTILITIES),703 HIGHWAY 8,31411,1986-04-26,Model,"703 highway 8, Hamilton, Ontario",43.212915,-79.698300,"703 Hwy 8, Stoney Creek, Ontario, L8E 5J6","(703 Hwy 8, Stoney Creek, Ontario, L8E 5J6, (4...",...,9.0,10.0,10,Jeff Beattie,905-546-2701,jeff.beattie@hamilton.ca,https://www.hamilton.ca/city-council/council-c...,https://www.hamilton.ca/sites/default/files/st...,3.139529e+07,31057.200763
3,ALEXANDER PARK - FIELDHOUSE,201 WHITNEY AVENUE,1954,1973-01-02,2017,"201 whitney avenue, Hamilton, Ontario",43.252779,-79.927670,"201 Whitney Ave, Hamilton, Ontario, L8S 2G6","(201 Whitney Ave, Hamilton, Ontario, L8S 2G6, ...",...,0.0,1.0,1,Maureen Wilson,905-546-2416,ward1@hamilton.ca,https://www.hamilton.ca/city-council/council-c...,https://www.hamilton.ca/sites/default/files/st...,1.532535e+07,27766.730529
4,ALEXANDER PARK WADING POOL UTILITY BUILDING,201 WHITNEY AVENUE,72,2011-01-01,2017,"201 whitney avenue, Hamilton, Ontario",43.252779,-79.927670,"201 Whitney Ave, Hamilton, Ontario, L8S 2G6","(201 Whitney Ave, Hamilton, Ontario, L8S 2G6, ...",...,0.0,1.0,1,Maureen Wilson,905-546-2416,ward1@hamilton.ca,https://www.hamilton.ca/city-council/council-c...,https://www.hamilton.ca/sites/default/files/st...,1.532535e+07,27766.730529
5,ANCASTER AQUATIC CENTRE (P),47 MEADOWBROOK DRIVE,19067,1982-01-02,2020,"47 meadowbrook drive, Hamilton, Ontario",43.215197,-80.006172,"47 Meadowbrook Dr, Ancaster, Ontario, L9G 4Y3","(47 Meadowbrook Dr, Ancaster, Ontario, L9G 4Y3...",...,11.0,12.0,12,Craig Cassar,905-546-2704,craig.cassar@hamilton.ca,https://www.hamilton.ca/city-council/council-c...,https://www.hamilton.ca/sites/default/files/st...,2.976397e+08,83212.280660
6,ANCASTER COMMUNITY CENTRE PARK - FIELDHOUSE,385 JERSEYVILLE ROAD WEST,1112,1996-01-02,2011,"385 jerseyville road west, Hamilton, Ontario",43.216734,-80.007322,"385 Jerseyville Rd W, Ancaster, Ontario, L9G 3L5","(385 Jerseyville Rd W, Ancaster, Ontario, L9G ...",...,11.0,12.0,12,Craig Cassar,905-546-2704,craig.cassar@hamilton.ca,https://www.hamilton.ca/city-council/council-c...,https://www.hamilton.ca/sites/default/files/st...,2.976397e+08,83212.280660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,WOODLAND CEMETERY EQUIPMENT STORAGE,700 SPRING GARDENS ROAD,1200,1923-04-02,NaN,"700 spring gardens road, Hamilton, Ontario",43.286247,-79.881668,"700 Spring Gardens Rd, Burlington, Ontario, L7...","(700 Spring Gardens Rd, Burlington, Ontario, L...",...,0.0,1.0,1,Maureen Wilson,905-546-2416,ward1@hamilton.ca,https://www.hamilton.ca/city-council/council-c...,https://www.hamilton.ca/sites/default/files/st...,1.532535e+07,27766.730529
495,WOODLAND CEMETERY SOIL STORAGE,700 SPRING GARDENS ROAD,600,2021-11-08,NaN,"700 spring gardens road, Hamilton, Ontario",43.286247,-79.881668,"700 Spring Gardens Rd, Burlington, Ontario, L7...","(700 Spring Gardens Rd, Burlington, Ontario, L...",...,0.0,1.0,1,Maureen Wilson,905-546-2416,ward1@hamilton.ca,https://www.hamilton.ca/city-council/council-c...,https://www.hamilton.ca/sites/default/files/st...,1.532535e+07,27766.730529
496,Woodlands Park - Concession / Washrooms,501 Barton St. E.,2495,1966-01-02,2020,"501 barton st. e., Hamilton, Ontario",43.259415,-79.844392,"501 Barton St E, Hamilton, Ontario, L8L 2Y8","(501 Barton St E, Hamilton, Ontario, L8L 2Y8, ...",...,2.0,3.0,3,Nrinder Nann,905-546-2702,ward3@hamilton.ca,https://www.hamilton.ca/city-council/council-c...,https://www.hamilton.ca/sites/default/files/st...,1.515768e+07,22796.903585
497,Woodward Park - Washroom / Sun Shelter,589 Woodward Ave.,667,1999-01-02,Audit,"589 woodward ave., Hamilton, Ontario",43.250085,-79.777260,"589 Woodward Ave, Hamilton, Ontario, L8H 6P2","(589 Woo

In [39]:
join.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 497 entries, 2 to 498
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   Asset Name               497 non-null    object  
 1   Asset Name: Address      497 non-null    object  
 2   Asset Name: Size         485 non-null    object  
 3   Asset Name: Date Built   490 non-null    object  
 4   Custom: Assessment Date  472 non-null    object  
 5   formattted_address       497 non-null    object  
 6   latitude                 497 non-null    float64 
 7   longitude                497 non-null    float64 
 8   formatted_address        497 non-null    object  
 9   location                 497 non-null    object  
 10  geometry                 497 non-null    geometry
 11  index_right              494 non-null    float64 
 12  OBJECTID                 494 non-null    float64 
 13  WARD                     494 non-null    object  
 14  CO

In [40]:
join.to_csv("facilities_wlocation.csv")

In [5]:
import json

def convert_geojson_to_json(geojson_file, output_json_file):
    with open(geojson_file, 'r') as f:
        geojson_data = json.load(f)
    
    # Convert GeoJSON to JSON
    json_data = json.dumps(geojson_data)
    
    # Write the JSON data to a file
    with open(output_json_file, 'w') as f:
        f.write(json_data)

# Example usage
geojson_file = 'Ward_Boundaries.geojson'
output_json_file = 'Ward_Boundaries.json'
convert_geojson_to_json(geojson_file, output_json_file)